# Get images from Kaggle

### Install Kaggle API client

In [ ]:
%%bash
pip install kaggle

In [ ]:
%%bash

mkdir /home/ec2-user/.kaggle/
mv /home/ec2-user/SageMaker/kaggle.json /home/ec2-user/.kaggle/
chmod 600 /home/ec2-user/.kaggle/kaggle.json

In [ ]:
%%bash

kaggle datasets list --user paultimothymooney --min-size 1

In [ ]:
%%bash

kaggle datasets download paultimothymooney/breast-histopathology-images

In [ ]:
%%bash

unzip breast-histopathology-images.zip -d images
rm -rf breast-histopathology-images.zip

In [ ]:
rm -rf images/IDC_*

In [ ]:
%%bash

mkdir images/0
mkdir images/1

In [ ]:
import os

In [ ]:
for path, subdirs, files in os.walk('images'):
    for name in files:
        filename = os.path.join(path, name)
        if name.endswith('class0.png'): 
            destination_class = '0'
        else:
            destination_class = '1'
        os.rename(filename, os.path.join('images', destination_class, name))

In [ ]:
%%bash
shopt -s extglob
cd images
rm -rf !("0"|"1")

# Explore the images

### Count the number of images for each class

In [ ]:
%%bash
cd images/0
ls -l | wc -l

In [ ]:
%%bash
cd images/1
ls -l | wc -l

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

%matplotlib inline

In [ ]:
img = mpimg.imread('images/0/8975_idx5_x2851_y1201_class0.png')
imgplot = plt.imshow(img)
plt.show()

In [ ]:
img = mpimg.imread('images/1/9075_idx5_x801_y801_class1.png')
imgplot = plt.imshow(img)
plt.show()

## Convert images to the RecordIO and upload to S3 

### Get the im2rec script from Apache MXNet Github repository

In [ ]:
%%bash
wget https://raw.githubusercontent.com/apache/incubator-mxnet/master/tools/im2rec.py 
chmod +x im2rec.py 

### Create the .lst files needed to creating the .rec files for train and test datasets 

In [ ]:
%%bash 
python im2rec.py --list --recursive --test-ratio 0.3 --train-ratio 0.7 images images/

### Create .rec files for train and test datasets

In [ ]:
%%bash

python im2rec.py --num-thread 4 --pass-through images_train.lst images
python im2rec.py --num-thread 4 --pass-through images_test.lst images

### Uploading the train and test .rec files to S3

In [ ]:
%%bash

aws s3 cp images_train.rec s3://sagemaker-data-mh/breast-cancer-detection/input/recordio/train/
aws s3 cp images_test.rec s3://sagemaker-data-mh/breast-cancer-detection/input/recordio/test/